In [2]:
from transformers import pipeline, set_seed

In [4]:
generator = pipeline('text-generation', model='gpt2')

Some weights of GPT2Model were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
generator('tripping on hallucinogenics', max_length=30, num_return_sequences=5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'tripping on hallucinogenics.\n\nMolly Shipp (Luxury House), a comedian with a reputation for playing the best ('},
 {'generated_text': 'tripping on hallucinogenics and caffeine.\n\nThe article states:\n\n"If you like to make a good first impression, caffeine'},
 {'generated_text': 'tripping on hallucinogenics. This is also what happened to the rest of the drugs that have not been reviewed.\n\nA lot of'},
 {'generated_text': 'tripping on hallucinogenics, cocaine, heroin, cannabis, methamphetamine, cocaine, methamphetamine, and mushrooms, and hallucinogenics also have'},
 {'generated_text': 'tripping on hallucinogenics, an idea already in development by some of the original LSD researchers. When the researcher discovered hallucinogenic compounds in'}]

In [9]:
from transformers import GPT2Tokenizer, GPT2Model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')

text = 'Tripping on hallucinogenics'
encoded_input = tokenizer(text, return_tensors='pt')

Some weights of GPT2Model were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
encoded_input

{'input_ids': tensor([[14824,  2105,   319, 23251,   259,  6644,   873]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}

In [13]:
output = model(**encoded_input)

In [17]:
encoded = tokenizer.encode(text)
encoded

[14824, 2105, 319, 23251, 259, 6644, 873]

In [20]:
for id in encoded:
    print(tokenizer.decode([id]))

Tri
pping
 on
 halluc
in
ogen
ics


In [21]:
import torch

In [36]:
t1 = torch.LongTensor(encoded)[None]
t1.shape

torch.Size([1, 7])

In [39]:
preds = model.generate(encoded_input['input_ids'])